In [3]:
import pandas as pd

In [4]:
column_names = ['id', 'series', 'date', 'query', 'user_id', 'twitters']

df_twitters = pd.read_csv('/home/borges/Documentos/Bases_dados/twitter/training.1600000.processed.noemoticon.csv',
                          encoding='latin1', names=column_names)

In [5]:
print(df_twitters.shape)
df_twitters.head(10)

(1600000, 6)


,id,series,date,query,user_id,twitters
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


# Subir os dados para o banco de dados

In [11]:
!pip install mysql-connector-python

  Using cached mysql_connector_python-8.0.33-cp311-cp311-manylinux1_x86_64.whl (27.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.6 MB/s eta 0:00:00 MB/s eta 0:00:01


In [12]:

import mysql.connector

def criar_conexao(host, usuario, password, banco):
    return mysql.connector.connect(
        host=host,
        user=usuario,
        password=password,
        database=banco
    )


def fechar_conexao(con):
    return con.close()


def insert_banco(id_tweets, series, date,
                 query, user_id, twitters):
    con = criar_conexao("localhost", "root", ", "twitter")
    cursor = con.cursor()
    
    sql = """INSERT INTO tweets (
             id_tweets, series, date,
             query, user_id, twitters) VALUES (
             %s, %s, %s, %s, %s, %s)
          """
    valores = (id_tweets, series, date,
               query, user_id, twitters)
    
    cursor.execute(sql, valores)

    cursor.close()
    con.commit()
    fechar_conexao(con)


In [13]:
# criar tabela se não existir

mydb = criar_conexao("localhost", "root", "", "twitter")

mycursor = mydb.cursor()

mycursor.execute("""CREATE TABLE IF NOT EXISTS tweets (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    id_tweets BIGINT,
                    series VARCHAR(255),
                    date VARCHAR(100),
                    query VARCHAR(100),
                    user_id VARCHAR(255),
                    twitters VARCHAR(255)
                    );
                """)